## this is stuff that I had already coded for but didnt present

In [ ]:
for group_name in unique_groups:
    group_peaks = [ppm for ppm, grp in identified_groups if grp == group_name]
    print(f"Simulating group: {group_name}, peaks: {group_peaks}")
    if group_peaks:
        if n_spins <= 4:
            I = np.eye(2)
            sz = np.array([[1, 0], [0, -1]]) / 2
            sx = np.array([[0, 1], [1, 0]]) / 2  # Pauli X operator
            Iz_ops = []
            Ix_ops = []
            for i in range(n_spins):
                op_z = 1
                op_x = 1
                for j in range(n_spins):
                    op_z = kron(op_z, sz if i == j else I)
                    op_x = kron(op_x, sx if i == j else I)
                Iz_ops.append(op_z)
                Ix_ops.append(op_x)
            # Ising (J-coupling) Hamiltonian
            H = 0
            for i in range(n_spins):
                for j in range(i + 1, n_spins):
                    J = j_couplings[0] if j_couplings else 7
                    H += 2 * np.pi * J * Iz_ops[i] @ Iz_ops[j]
            # Add transverse field (X) term for dynamics
            Bx = 5  # adjust field strength for more/less mixing
            for i in range(n_spins):
                H += 2 * np.pi * Bx * Ix_ops[i]
            eigvals, eigvecs = np.linalg.eigh(H)
            print_quantum_simulation_results(group_name, H, eigvals, eigvecs)

            # Animation code for each group
            import matplotlib.pyplot as plt
            from matplotlib.animation import FuncAnimation
            from IPython.display import HTML

            psi0 = np.zeros(H.shape[0], dtype=complex)
            psi0[0] = 1.0
            times = np.linspace(0, 1, 200)
            c = np.dot(eigvecs.conj().T, psi0)
            def psi_t(t):
                phase = np.exp(-1j * eigvals * t)
                return np.dot(eigvecs, c * phase)
            fig, ax = plt.subplots(figsize=(8, 4))
            lines = []
            for i in range(H.shape[0]):
                (line,) = ax.plot([], [], label=f'State {i}')
                lines.append(line)
            ax.set_xlim(times[0], times[-1])
            ax.set_ylim(0, 1)
            ax.set_xlabel('Time')
            ax.set_ylabel('Probability')
            ax.set_title(f'Spin System Time Evolution: {group_name}')
            ax.legend()
            def animate(frame):
                t = times[frame]
                psi = psi_t(t)
                for i, line in enumerate(lines):
                    line.set_data(times[:frame+1], [np.abs(psi_t(tt)[i])**2 for tt in times[:frame+1]])
                return lines
            ani = FuncAnimation(fig, animate, frames=len(times), interval=50, blit=True)
            plt.show()
            display(HTML(ani.to_jshtml()))
        